In [78]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil

In [82]:
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('^N225', 'yahoo', start, end)
# data = web.DataReader('AAPL', 'yahoo', start, end)
data = data.astype("double")
print(data)

initial = 1000000
fname = f"html/plot-"

                  High         Low        Open       Close       Volume  \
Date                                                                      
2021-09-13  151.419998  148.750000  150.630005  149.550003  102404300.0   
2021-09-14  151.070007  146.910004  150.350006  148.119995  109296300.0   
2021-09-15  149.440002  146.369995  148.559998  149.029999   83281300.0   
2021-09-16  148.970001  147.220001  148.440002  148.789993   68034100.0   
2021-09-17  148.820007  145.759995  148.820007  146.059998  129868800.0   
...                ...         ...         ...         ...          ...   
2022-09-02  160.360001  154.970001  159.750000  155.809998   76905200.0   
2022-09-06  157.089996  153.690002  156.470001  154.529999   73714800.0   
2022-09-07  156.669998  153.610001  154.820007  155.960007   87449600.0   
2022-09-08  156.360001  152.679993  154.639999  154.460007   84923800.0   
2022-09-09  157.820007  154.750000  155.470001  157.369995   68028800.0   

             Adj Close  

**MACD CROSS**

In [77]:
import BacktestingStrategies.Strategy_MacdCross as btstMACD
importlib.reload(btstMACD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACD.MACDCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）

#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)
output2=bt.optimize(\
    n1=range(10, 25, 5),\
    n2=range(10, 75, 10),\
    ns=range(10, 15, 5),\
    maximize='Equity Final [$]',
    method='grid',\
    constraint = lambda x: (x.n2 > x.n1) & (x.n1 > x.ns),\
    max_tries = 100)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-13 00:00:00
End                       2022-09-09 00:00:00
Duration                    361 days 00:00:00
Exposure Time [%]                   41.434263
Equity Final [$]                  1135.753628
Equity Peak [$]                   1177.633603
Return [%]                          13.575363
Buy & Hold Return [%]                5.229015
Return (Ann.) [%]                   13.632978
Volatility (Ann.) [%]               19.321522
Sharpe Ratio                         0.705585
Sortino Ratio                        1.182371
Calmar Ratio                         0.782572
Max. Drawdown [%]                  -17.420733
Avg. Drawdown [%]                   -4.370935
Max. Drawdown Duration      245 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                    5
Win Rate [%]                             60.0
Best Trade [%]                      20.593488
Worst Trade [%]                    -12.302583
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,6,47,68,154.249781,171.139999,101.341310,0.109499,2021-11-17,2021-12-17,30 days
1,6,98,110,175.474320,168.880005,-39.565891,-0.037580,2022-02-01,2022-02-17,16 days
2,6,130,145,164.791697,170.089996,31.789798,0.032151,2022-03-18,2022-04-08,21 days
3,7,179,189,150.380717,131.880005,-129.504988,-0.123026,2022-05-27,2022-06-13,17 days
4,6,196,237,138.954441,167.570007,171.693399,0.205935,2022-06-23,2022-08-22,60 days


Row(id='50904', ...)

**EMA CROSS**

In [71]:
import BacktestingStrategies.Strategy_EMACross as btstEMA
importlib.reload(btstEMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstEMA.EmaCrossStrategy, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") 

#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    n1=range(10, 100, 10),\
    n2=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 100)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-13 00:00:00
End                       2022-09-09 00:00:00
Duration                    361 days 00:00:00
Exposure Time [%]                    54.98008
Equity Final [$]                  1175.163235
Equity Peak [$]                    1275.06483
Return [%]                          17.516324
Buy & Hold Return [%]                5.229015
Return (Ann.) [%]                   17.591917
Volatility (Ann.) [%]               21.225829
Sharpe Ratio                         0.828798
Sortino Ratio                        1.477905
Calmar Ratio                         1.134002
Max. Drawdown [%]                  -15.513123
Avg. Drawdown [%]                   -3.259228
Max. Drawdown Duration      212 days 00:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                    8
Win Rate [%]                             50.0
Best Trade [%]                      17.679518
Worst Trade [%]                     -4.637589
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,6,24,42,145.556954,147.919998,14.178263,0.016234,2021-10-15,2021-11-10,26 days
1,6,46,81,151.747450,172.000000,121.515300,0.133462,2021-11-16,2022-01-06,51 days
2,6,97,107,175.645160,168.880005,-40.590929,-0.038516,2022-01-31,2022-02-14,14 days
3,6,108,110,173.645304,168.880005,-28.591793,-0.027443,2022-02-15,2022-02-17,2 days
4,6,130,145,164.791697,170.089996,31.789798,0.032151,2022-03-18,2022-04-08,21 days
5,7,180,187,149.576754,142.639999,-48.557285,-0.046376,2022-05-31,2022-06-09,9 days
6,7,197,239,142.361221,167.529999,176.181447,0.176795,2022-06-24,2022-08-24,61 days
7,7,240,241,170.871647,163.619995,-50.761565,-0.042439,2022-08-25,2022-08-26,1 days


Row(id='46146', ...)

**SMA**

In [64]:
import BacktestingStrategies.Strategy_SMACross as btstSMA
importlib.reload(btstSMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMA.SmaCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    n1=range(10, 100, 5),\
    n2=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 100)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-13 00:00:00
End                       2022-09-09 00:00:00
Duration                    361 days 00:00:00
Exposure Time [%]                   27.091633
Equity Final [$]                     915.3018
Equity Peak [$]                        1000.0
Return [%]                           -8.46982
Buy & Hold Return [%]                5.229015
Return (Ann.) [%]                   -8.502087
Volatility (Ann.) [%]               11.118805
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -15.014826
Avg. Drawdown [%]                  -15.014826
Max. Drawdown Duration      212 days 00:00:00
Avg. Drawdown Duration      212 days 00:00:00
# Trades                                    3
Win Rate [%]                        33.333333
Best Trade [%]                       6.897418
Worst Trade [%]                     -9.642861
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,5,104,113,177.152585,160.070007,-85.412887,-0.096429,2022-02-09,2022-02-23,14 days
1,5,136,153,176.469226,166.419998,-50.246140,-0.056946,2022-03-28,2022-04-21,24 days
2,5,206,245,147.767841,157.960007,50.960827,0.068974,2022-07-08,2022-09-01,55 days


Row(id='38124', ...)

**ボリンジャーバンド**

In [66]:
import BacktestingStrategies.Strategy_BBD as btstBBD
importlib.reload(btstBBD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBD.BBsigma, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") 

Start                     2021-09-13 00:00:00
End                       2022-09-09 00:00:00
Duration                    361 days 00:00:00
Exposure Time [%]                   44.621514
Equity Final [$]                  1003.969432
Equity Peak [$]                   1053.598811
Return [%]                           0.396943
Buy & Hold Return [%]                5.229015
Return (Ann.) [%]                    0.398528
Volatility (Ann.) [%]               22.922907
Sharpe Ratio                         0.017386
Sortino Ratio                        0.024515
Calmar Ratio                         0.017461
Max. Drawdown [%]                  -22.823353
Avg. Drawdown [%]                   -7.854014
Max. Drawdown Duration      128 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                   17
Win Rate [%]                        17.647059
Best Trade [%]                      24.334565
Worst Trade [%]                     -6.892514
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,5,89,90,167.052834,164.509995,-12.714198,-0.015222,2022-01-19,2022-01-20,1 days
1,5,90,91,165.324319,162.410004,-14.571577,-0.017628,2022-01-20,2022-01-21,1 days
2,5,91,92,163.213933,161.619995,-7.969690,-0.009766,2022-01-21,2022-01-24,3 days
3,5,92,93,162.420014,159.779999,-13.200077,-0.016254,2022-01-24,2022-01-25,1 days
4,5,93,122,160.570910,157.440002,-15.654537,-0.019499,2022-01-25,2022-03-08,42 days
5,5,122,125,158.219330,154.729996,-17.446674,-0.022054,2022-03-08,2022-03-11,3 days
6,5,125,126,155.495909,150.619995,-24.379570,-0.031357,2022-03-11,2022-03-14,3 days
7,5,126,134,151.365564,174.070007,113.522216,0.149997,2022-03-14,2022-03-24,10 days
8,6,156,157,157.576163,156.570007,-6.036934,-0.006385,2022-04-26,2022-04-27,1 days
9,6,157,167,157.345029,146.500000,-65.070173,-0.068925,2022-04-27,2022-05-11,14 days


Row(id='40247', ...)

**RSI**

In [83]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=True, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 100)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-13 00:00:00
End                       2022-09-09 00:00:00
Duration                    361 days 00:00:00
Exposure Time [%]                    13.14741
Equity Final [$]               1113249.636527
Equity Peak [$]                1113249.636527
Return [%]                          11.324964
Buy & Hold Return [%]                5.229015
Return (Ann.) [%]                   11.372557
Volatility (Ann.) [%]                7.035468
Sharpe Ratio                         1.616461
Sortino Ratio                        3.738795
Calmar Ratio                         3.692088
Max. Drawdown [%]                   -3.080251
Avg. Drawdown [%]                    -1.76764
Max. Drawdown Duration       20 days 00:00:00
Avg. Drawdown Duration       12 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       11.32619
Worst Trade [%]                      11.32619
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,7051,16,48,141.808495,157.869995,113249.636527,0.113262,2021-10-05,2021-11-18,44 days


Row(id='53314', ...)